In [ ]:
using Revise

using Statistics
using Symbolics
using ArrayOperations

import Base: getindex,
             @propagate_inbounds

In [ ]:
const δ⁻ = Backward{1}(-), Backward{2}(-)
const δ⁺ = Forward{1}(-), Forward{2}(-)

const σ⁻ = Backward{1}(middle), Backward{2}(middle)
const σ⁺ = Forward{1}(middle), Forward{2}(middle);

In [ ]:
# \invw
function grad(ω, γ, a, b, ʍ)
    (δ⁻[1](ω) * b[1] + (σ⁺[1](a[1]) - b[1]) * γ - σ⁻[1](δ⁺[1](a[1]) * γ)) * ʍ[1],
    (δ⁻[2](ω) * b[2] + (σ⁺[2](a[2]) - b[2]) * γ - σ⁻[2](δ⁺[2](a[2]) * γ)) * ʍ[2]
end

function div(ω, γ, a, b)
    δ⁺[1](ω[1]) * a[1] + (σ⁻[1](b[1]) - a[1]) * γ[1] - σ⁺[1](δ⁻[1](b[1]) * γ[1]) +
    δ⁺[2](ω[2]) * a[2] + (σ⁻[2](b[2]) - a[2]) * γ[2] - σ⁺[2](δ⁻[2](b[2]) * γ[2])
end

In [ ]:
dims = 1, 1

@variables ω[dims...] γ[dims...]
@variables ax[dims...] ay[dims...]; a = (ax, ay)
@variables bx[dims...] by[dims...]; b = (bx, by)
@variables ʍx[dims...] ʍy[dims...]; ʍ = (ʍx, ʍy)

In [ ]:
qω = grad(ω, γ, a, b, ʍ)
qγ = qω

lap = div(qω, qγ, a, b)

In [ ]:
@syms i::Int j::Int

qωx, qωy = qω

string(qωx[i, j])

In [ ]:
Lij = lap[i, j]

string(Lij)

In [ ]:
Symbolics.get_variables(Lij)

In [ ]:
∂ωij = Differential(ω[i, j])

In [ ]:
∂Lij∂ωij = simplify(expand_derivatives(∂ωij(Lij)))

string(∂Lij∂ωij)

# Jacobian

In [ ]:
import ArrayOperations: OperatorSupport,
                        Stencil

const ∇₁ = ∇{Tuple{1}}()
const ∇₂ = ∇{Tuple{2}}()
const ∇₃ = ∇{Tuple{3}}()

The example below illustrates how to encode all the information relative to:

1. A ternary operator that is,
2. Multilinear with respect to its first two arguments,
3. Does not depend on the third.

In addition, instead of declaring the routines
```julia
@inline @propagate_inbounds function getindex(this::Ret{<:Jac{1, TerOp}}, i::Int, j::Int)
    op = operator(this)
    _, y, _ = arguments(this)

    ifelse(isequal(i, j), y[i-1], zero(y[i-1]))
end
```
and
```julia
@inline @propagate_inbounds function getindex(this::Ret{<:Jac{2, TerOp}}, i::Int, j::Int)
    op = operator(this)
    x, _, _ = arguments(this)

    ifelse(isequal(i-1, j), x[i], zero(x[i]))
end
```
to compute the coefficients of the jacobian with respect to the first and second arguments (which are diagonal and lower diagonal, respectively), the structure is encoded using the `OperatorSupport` and `Stencil` traits.


In [ ]:
struct TerOp <: Ary{3} end

@inline @propagate_inbounds function getindex(this::Ret{TerOp}, i::Int)
    op = operator(this)
    x, y, _ = arguments(this)

    x[i] * y[i-1]
end

# jacobian w.r.t. 1st argument is diagonal

OperatorSupport(::Type{<:Ret{<:Jac{1,TerOp}}}) = HasStencil()

Stencil(::Type{<:Ret{<:Jac{1,TerOp}}}) = Singleton{Tuple{0}}()

@inline @propagate_inbounds function getindex(this::Ret{<:Jac{1, TerOp}}, ::Type{Tuple{0}}, (j,)::Tuple{Int})
    op = operator(this)
    _, y, _ = arguments(this)

    y[j-1]
end

# jacobian w.r.t. 2nd argument is lower diagonal

OperatorSupport(::Type{<:Ret{<:Jac{2,TerOp}}}) = HasStencil()

Stencil(::Type{<:Ret{<:Jac{2,TerOp}}}) = Singleton{Tuple{-1}}()

@inline @propagate_inbounds function getindex(this::Ret{<:Jac{2, TerOp}}, ::Type{Tuple{-1}}, (j,)::Tuple{Int})
    op = operator(this)
    x, _, _ = arguments(this)

    x[j+1]
end

# jacobian w.r.t. 3rd argument is null

OperatorSupport(::Type{<:Ret{<:Jac{3,TerOp}}}) = NullSupport()

There is no use case for this for the time being, but one could envision define the dependences of higher-order derivatives as follows.

In [ ]:
OperatorSupport(::Type{<:Ret{<:Hess{1,1,TerOp}}}) = NullSupport()
OperatorSupport(::Type{<:Ret{<:Hess{2,1,TerOp}}}) = NullSupport()
OperatorSupport(::Type{<:Ret{<:Hess{1,2,TerOp}}}) = NullSupport()
OperatorSupport(::Type{<:Ret{<:Hess{2,2,TerOp}}}) = NullSupport()

In [ ]:
f₃ = TerOp()

d₁f₃ = ∇₁(f₃)
d₂f₃ = ∇₂(f₃)
d₃f₃ = ∇₃(f₃)

In [ ]:
n = 32; x = rand(32); y = rand(32); z = rand(32);

In [ ]:
f₀ = f₃(x, y, z)

d₁f₀ = d₁f₃(x, y, z)
d₂f₀ = d₂f₃(x, y, z)
d₃f₀ = d₃f₃(x, y, z)

In [ ]:
i = 4

@assert isequal(f₀[i], x[i] * y[i-1])
@assert isequal(d₁f₀[i, i-1], zero(y[i-1]))
@assert isequal(d₁f₀[i, i], y[i-1])
@assert isequal(d₁f₀[i, i+1], zero(y[i-1]))
@assert isequal(d₂f₀[i, i-1], x[i])
@assert isequal(d₂f₀[i, i], zero(x[i]))
@assert isequal(d₂f₀[i, i+1], zero(x[i]))

In [ ]:
using BenchmarkTools

@btime $d₃f₀[$i, $i]
@btime $d₂f₀[$i, $i]
@btime $d₁f₀[$i, $i]

In [ ]:
@btime $x[$i]

In [ ]:
@btime $f₃($x, $y, $z)